# Test OPTIMADE Gateway on Heroku

This notebook outlines how one can perform OPTIMADE searches through the OPTIMADE Gateway deployed on Heroku.

In [2]:
import os

os.environ["OPTIMADE_CONFIG_FILE"] = "../../config_heroku.yml"

import json
from optimade.adapters import Structure
from optimade_gateway.models import DatabasesResponse, QueriesResponseSingle
from pydantic import ValidationError
from random import randint
import requests

OPTIMADE_GATEWAY_BASE_URL = "https://optimade-gateway.herokuapp.com"

## Retrieve list of known OPTIMADE databases

Let's check what databases are available.

In [ ]:
response = requests.get(f"{OPTIMADE_GATEWAY_BASE_URL}/databases")
if response.ok:
    databases_response = DatabasesResponse(**response.json())
else:
    raise RuntimeError(
        f"Unsuccessful response.\nStatus code: {response.status_code}\n"
        f"Response:\n{response.json()}"
    )

print("Available databases:")
print(
    "\n".join(
        f"- {_.id} ({_.attributes.base_url})"
        for _ in databases_response.data
    )
)

## Define OPTIMADE search

Use the database ID (e.g., `mcloud/sssp`) in the list of `database_ids` when using the `/search` endpoint.
Otherwise, simply use the [standard OPTIMADE `/structures` query parameters](https://github.com/Materials-Consortia/OPTIMADE/blob/develop/optimade.rst#entry-listing-url-query-parameters).

In [ ]:
database_ids = [
    "mcloud/sssp",
    "cod",
    "mp",
    "mcloud/mc3d-structures",
]

optimade_filter = 'elements HAS ALL "Si","Al","Mg"'

In [ ]:
response = requests.get(
    f"{OPTIMADE_GATEWAY_BASE_URL}/search",
    params={
        "database_ids": database_ids,
        "filter": optimade_filter,
        "timeout": 30,
    }
)
if response.ok:
    search_response = QueriesResponseSingle(**response.json())
else:
    raise RuntimeError(
        f"Unsuccessful response.\nStatus code: {response.status_code}\n"
        f"Response:\n{json.dumps(response.json(), indent=2)}"
    )

print("Query overview:")
print(
    "\n".join(
        f"- {id_} (Returned results: {len(data)})"
        for id_, data in search_response.data.attributes.response.data.items()
    )
)

## Curate and process search results

In [ ]:
structures = {}
for (
    database, database_structures
) in search_response.data.attributes.response.data.items():
    structures[database] = []
    for structure in database_structures:
        try:
            structures[database].append(Structure(structure.dict()))
        except ValidationError:
            continue

print("\nQuery overview (after validating structure properties):")
print(
    "\n".join(
        f"- {id_} (Valid results: {len(data)})"
        for id_, data in structures.items()
    )
)

The `structures` dictionary now holds all the structures that validate against the [OPTIMADE Python tools](https://github.com/Materials-Consortia/optimade-python-tools) structure-type model.
The keys of the `structures` dictionary are the database IDs, while the values are a list of [OPTIMADE Python tools `Structure` adapters](https://www.optimade.org/optimade-python-tools/latest/api_reference/adapters/structures/adapter/#optimade.adapters.structures.adapter.Structure), with which one can convert the structure to other well-known formats, e.g., ASE `Atoms`, AiiDA `StructureData` or even a CIF file or PDB file.
For the latter, it will return the string content for such a file, you will have to write it to a file object yourself.

The following is a showcase of converting a random structure in `structures` to a CIF file syntax:

In [ ]:
for db_structures in structures.values():
    if db_structures:
        a_structure = db_structures[randint(0, len(db_structures)-1)]
print(a_structure.as_cif)